In [ ]:
from appgeopy import *
from my_packages import *

#### Average GWR Coefficients (slope + intercept)

In [ ]:
savefld = "GWR_Average_Coeffs"

fpath = r"GWR_Prediction_Output/GWR_Layer_4.feather"
layer = os.path.basename(fpath).split(".")[0][4:]

df = pd.read_feather(fpath)
datetime_arr = pd.to_datetime([ele[1:] for ele in df["datetime_str"]])
df.insert(
    loc=df.columns.get_loc("datetime_str"),
    column="datetime",
    value=datetime_arr,
)
show(df)

**convert the dataframe into netcdf data type (I call it geocube)**

**to view the structure, just use `data_cube` built-in function**

In [ ]:
df_cube = DataCube.from_dataframe(
    df=df, time_col="datetime", x_col="X_TWD97", y_col="Y_TWD97"
)
df_cube.data_cube

**get the average value of DIFFDISP coefficient timeseries**

**get the average value of DIFFDISP intercept timeseries**

In [ ]:
intercept_cube = df_cube.data_cube["Intercept_coef"]

mean_intercept = intercept_cube.mean(dim="datetime")
mean_intercept.plot(cmap="tab20")
plt.show()

In [ ]:
stdev_intercept = intercept_cube.std(dim="datetime")
stdev_intercept.plot(cmap="tab20c")
plt.show()

In [ ]:
# export to netcdf
mean_intercept.to_netcdf(
    os.path.join(savefld, f"GWR_AverageIntercept_{layer}.nc")
)
stdev_intercept.to_netcdf(
    os.path.join(savefld, f"GWR_StdevIntercept_{layer}.nc")
)

In [ ]:
# FIX: Using the correct, dictionary-based structure for styling parameters
# ----------------------------------------------------------------------
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
from geodatasets import get_path
from matplotlib_map_utils.core.north_arrow import north_arrow
from matplotlib_scalebar.scalebar import ScaleBar

# Data Setup (The Fix from the first error)
path_to_nybb = get_path("nybb")
nyc_boroughs = gpd.read_file(path_to_nybb).to_crs(epsg=32618)


# --- Plotting ---
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
nyc_boroughs.plot(ax=ax, color="lightgray", edgecolor="black", linewidth=0.5)
ax.set_xlabel("Easting (m)")
ax.set_ylabel("Northing (m)")
ax.set_title("NYC Boroughs (Projected to UTM 18N)")

# --- Scale Bar ---
scalebar = ScaleBar(
    dx=1,
    units="m",
    location="lower right",
    box_alpha=0.6,
    frameon=False,
    border_pad=0.5,
    length_fraction=0.2,
)
ax.add_artist(scalebar)

# --- North Arrow (Fix cho lỗi CRS) ---
# north_arrow(
#     ax=ax,
#     location='upper right',
#     label={'text': 'N', 'fontsize': 16},
#     scale=0.5,
#     rotation={'crs': pyproj.CRS.from_user_input(32618), 'degrees': 0}
# )

# --- North Arrow (Sửa lỗi 'base_color' và Tùy biến kiểu dáng) ---
north_arrow(
    ax=ax,
    location="upper right",
    scale=0.5,
    rotation={"degrees": 0},
    # 1. Định nghĩa phần thân mũi tên (base)
    # FIX: Dùng 'facecolor' và 'edgecolor' thay cho 'color'
    base={
        "facecolor": "lightgray",  # Màu nền thân mũi tên
        "edgecolor": "black",  # Đường viền
        "lw": 1,  # Độ dày đường viền
    },
    # 2. Định nghĩa phần đầu nhọn (fancy)
    fancy={
        "facecolor": "darkred",  # Màu nền đầu mũi tên
        "edgecolor": "darkred",  # Màu viền đầu mũi tên
        "head_width": 1.5,
        "head_length": 2.0,
    },
    label={"text": "N", "fontsize": 24, "color": "black", "fontweight": "bold"},
)

plt.show()